In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

!pip install --upgrade xgboost
!pip install --upgrade scikit-learn
import xgboost as xgb

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
!cp /content/drive/MyDrive/2-folder/kaggle/df_utils.py /content/
import df_utils

In [3]:
def preprocess(df):
    df = df.copy()

    # def normalize_name(x):
    #     return " ".join([v.strip(",()[].\"'") for v in x.split(" ")])

    def ticket_number(x):
        return x.split(" ")[-1]

    def ticket_item(x):
        items = x.split(" ")
        if len(items) == 1:
            return "NONE"
        return "_".join(items[0:-1])

    # df["Name"] = df["Name"].apply(normalize_name)
    df["Ticket_number"] = df["Ticket"].apply(ticket_number)
    df["Ticket_item"] = df["Ticket"].apply(ticket_item)
    return df

In [10]:
df_train = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/titanic/train.csv')
print(df_train.head(5))

# df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch']
# df_train['IsAlone'] = (df_train['FamilySize'] == 0).astype(int)
# df_train['Title'] = df_train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
# common_titles = ['Mr', 'Miss', 'Mrs', 'Master']
# df_train['Title'] = df_train['Title'].apply(lambda x: x if x in common_titles else 'Other')
# title_map = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Other': 4}
# df_train['Title'] = df_train['Title'].map(title_map)
# print(df_train.head(5))

df_train_x = df_train.drop(['PassengerId', 'Name', 'Cabin'], axis=1)

# Categorical values to numeric
df_train_x = df_utils.one_hot_encoding(df_train_x, "Sex")
label_encoder = preprocessing.LabelEncoder()
df_train_x['Embarked']= label_encoder.fit_transform(df_train_x['Embarked'])

df_train_x = preprocess(df_train_x)
df_train_x = df_train_x.drop(['Ticket'], axis=1)
df_train_x['Ticket_item']= label_encoder.fit_transform(df_train_x['Ticket_item'])
df_train_x.Ticket_number = pd.to_numeric(df_train_x.Ticket_number, errors='coerce')

df_train_x = df_train_x.dropna()
df_train_x = df_train_x.reset_index(drop=True)

# Fix NaN
# print(df_train_x.isna().any())
# df_utils.fill_with_mean("Age", df_train_x)
# df_utils.fill_with_mode("Embarked", df_train_x)
# df_utils.fill_with_mean("Ticket_number", df_train_x)

# print(f"df_train_x.shape[0] = {df_train_x.shape[0]}")
# print(df_train_x.head(10))

# ----------------------------------------------------------------

y = df_train_x['Survived'].values
# print(y[0:5])
df_train_x = df_train_x.drop(['Survived'], axis=1)

# normalize dataframe
# df_train_x=(df_train_x-df_train_x.mean())/df_train_x.std()

X = df_train_x.values  #.astype(float)
# print(X[0:5])

X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.1, random_state=3)
print('Shape of X training set {}'.format(X_trainset.shape),'&',' Size of Y training set {}'.format(y_trainset.shape))
print('Shape of X test set {}'.format(X_testset.shape),'&','Size of y test set {}'.format(y_testset.shape))

# Addestra il modello
dtrain = xgb.DMatrix(X_trainset, label=y_trainset)
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}
model = xgb.train(params, dtrain, num_boost_round=100)

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
Sh

In [19]:
df_test = pd.read_csv('/content/drive/MyDrive/2-folder/kaggle/titanic/test.csv')
#print(df_train.head(10))

df_test_x = df_test.drop(['PassengerId', 'Name', 'Cabin'], axis=1)

# Categorical values to numeric
df_test_x = df_utils.one_hot_encoding(df_test_x, "Sex")
label_encoder = preprocessing.LabelEncoder()
df_test_x['Embarked']= label_encoder.fit_transform(df_test_x['Embarked'])

df_test_x = preprocess(df_test_x)
df_test_x = df_test_x.drop(['Ticket'], axis=1)
df_test_x['Ticket_item']= label_encoder.fit_transform(df_test_x['Ticket_item'])
df_test_x.Ticket_number = pd.to_numeric(df_test_x.Ticket_number, errors='coerce')

# Fix NaN
# print(df_test_x.isna().any())
df_utils.fill_with_mean("Age", df_test_x)
df_utils.fill_with_mean("Fare", df_test_x)

# print(f"df_test_x.shape[0] = {df_test_x.shape[0]}")
print(df_test_x.head(10))

# normalize dataframe
# df_test_x=(df_test_x-df_test_x.mean())/df_test_x.std()

X = df_test_x.values  #.astype(float)
print(X[0:5])

X = xgb.DMatrix(X)
y_ = model.predict(X)
print(y_[0:5])

y_ = y_ > 0.5
print(y_[:5])
y_ = [1 if a_ > 0.5 else 0 for a_ in y_]
print(y_[:5])

predictions = pd.DataFrame({
        "PassengerId": df_test["PassengerId"],
        "Survived": y_
    })

def make_submission(predictions):
    path="/content/drive/MyDrive/2-folder/kaggle/titanic/submission.csv"
    predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

make_submission(predictions)

   Pclass   Age  SibSp  Parch     Fare  Embarked  Sex_female  Sex_male  \
0       3  34.5      0      0   7.8292         1         0.0       1.0   
1       3  47.0      1      0   7.0000         2         1.0       0.0   
2       2  62.0      0      0   9.6875         1         0.0       1.0   
3       3  27.0      0      0   8.6625         2         0.0       1.0   
4       3  22.0      1      1  12.2875         2         1.0       0.0   
5       3  14.0      0      0   9.2250         2         0.0       1.0   
6       3  30.0      0      0   7.6292         1         1.0       0.0   
7       2  26.0      1      1  29.0000         2         0.0       1.0   
8       3  18.0      0      0   7.2292         0         1.0       0.0   
9       3  21.0      2      0  24.1500         2         0.0       1.0   

   Ticket_number  Ticket_item  
0         330911           15  
1         363272           15  
2         240276           15  
3         315154           15  
4        3101298         